In [118]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Q1

In [119]:
def get_all_headers(url):

  data = []
  request = requests.get(url)
  Soup = BeautifulSoup(request.text, 'lxml')
  
  heading_tags = ["h"+str(i) for i in range(1,7)]
  for tags in Soup.find_all(heading_tags):
    row = {
        'Tag Name': tags.name,
        'Text': ' '.join(tags.text.strip(' ').split('\n')).rstrip(' ')
    }
    data.append(row)

  df = pd.DataFrame(data)
  print(df.to_string())

get_all_headers('https://en.wikipedia.org/wiki/Main_Page')

   Tag Name                           Text
0        h1                      Main Page
1        h1           Welcome to Wikipedia
2        h2  From today's featured article
3        h2               Did you know ...
4        h2                    In the news
5        h2                    On this day
6        h2       Today's featured picture
7        h2       Other areas of Wikipedia
8        h2    Wikipedia's sister projects
9        h2            Wikipedia languages
10       h2                Navigation menu
11       h3                 Personal tools
12       h3                     Namespaces
13       h3                          Views
14       h3                         Search
15       h3                     Navigation
16       h3                     Contribute
17       h3                          Tools
18       h3                   Print/export
19       h3              In other projects
20       h3                      Languages


## Q2, Q3

In [120]:
def get_top_100_movies(url):

  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")

  data = []

  for i in range(100):
    row = {
        'Name': soup.select('td.titleColumn')[i].get_text().split('\n')[2].lstrip(' '),
        'Rating': soup.select('td.posterColumn span[name=ir]')[i].attrs.get('data-value')[:4],
        'Year': soup.select('td.titleColumn')[i].get_text().split('\n')[3][1:-1]
    }
    data.append(row)

  df = pd.DataFrame(data)
  print(df.to_string())

get_top_100_movies('https://www.imdb.com/chart/top')
print("==============================================================================")
get_top_100_movies('https://www.imdb.com/india/top-rated-indian-movies/')

                                                                    Name Rating  Year
0                                               The Shawshank Redemption   9.23  1994
1                                                          The Godfather   9.15  1972
2                                                        The Dark Knight   8.98  2008
3                                                  The Godfather Part II   8.98  1974
4                                                           12 Angry Men   8.94  1957
5                                                       Schindler's List   8.93  1993
6                          The Lord of the Rings: The Return of the King   8.92  2003
7                                                           Pulp Fiction   8.85  1994
8                      The Lord of the Rings: The Fellowship of the Ring   8.80  2001
9                                         The Good, the Bad and the Ugly   8.79  1966
10                                                    

## Q4

In [121]:
def get_list_of_presidents(url):

  data = []

  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  for pres in soup.find_all(class_ = "presidentListing"):
    row = {
        'Name': pres.find(["h3"]).text.split(' (')[0],
        'Term of Office': pres.find("p").text.lstrip('Term of Office: ').rstrip(' ')
    }
    data.append(row)

  df = pd.DataFrame(data)
  print(df.to_string())

get_list_of_presidents('https://presidentofindia.nic.in/former-presidents.htm')

                            Name                                                       Term of Office
0           Shri Ram Nath Kovind                                       25 July, 2017 to 25 July, 2022
1          Shri Pranab Mukherjee                                       25 July, 2012 to 25 July, 2017
2   Smt Pratibha Devisingh Patil                                       25 July, 2007 to 25 July, 2012
3         DR. A.P.J. Abdul Kalam                                       25 July, 2002 to 25 July, 2007
4           Shri K. R. Narayanan                                       25 July, 1997 to 25 July, 2002
5        Dr Shankar Dayal Sharma                                       25 July, 1992 to 25 July, 1997
6            Shri R Venkataraman                                       25 July, 1987 to 25 July, 1992
7               Giani Zail Singh                                       25 July, 1982 to 25 July, 1987
8      Shri Neelam Sanjiva Reddy                                       25 July, 19

## Q5, Q6

In [122]:
def get_top_icc_team_details(url):

  response = requests.get(url)
  index = 1 if 'mens' in url.split('/') else 8
  teams_url = BeautifulSoup(response.text, "html.parser").find_all(class_="rankings-menu__link ", href=True)[index]['href']
  team_response = requests.get(url[:27]+teams_url)

  team_data = []
  top_team_info = soup = BeautifulSoup(team_response.text, "html.parser").find(class_ = "rankings-block__banner")
  row = {
      'Team': top_team_info.find(class_="u-hide-phablet").text,
      'Matches': top_team_info.find(class_="rankings-block__banner--matches").text,
      'Points': top_team_info.find(class_="rankings-block__banner--points").text,
      'Rating': top_team_info.find(class_="rankings-block__banner--rating u-text-right").text.strip('\n').strip(' ').strip('\n')
  }
  team_data.append(row)

  soup = BeautifulSoup(team_response.text, "html.parser").find_all(class_ = "table-body")
  for team in soup:
    row = {
        'Team': team.find(class_="table-body__cell rankings-table__team").text.split('\n')[2],
        'Matches': team.find_all(class_="table-body__cell u-center-text")[0].text,
        'Points': team.find_all(class_="table-body__cell u-center-text")[1].text,
        'Rating': team.find(class_="table-body__cell u-text-right rating").text
    }
    team_data.append(row)

  df = pd.DataFrame(team_data)
  print(df.to_string())
  
  batting_data = []
  soup = BeautifulSoup(response.text, "html.parser").find_all(class_ = "table rankings-card-table")
  top_player_info = BeautifulSoup(response.text, "html.parser").find_all(class_= "rankings-block__banner--player-info")[0]
  row = {
      'Rank': 1,
      'Name': top_player_info.find(class_="rankings-block__banner--name").text.strip('\n').strip(' '),
      'Country': top_player_info.find(class_="rankings-block__banner--nationality").text.strip('\n').strip(' '),
      'Rating': BeautifulSoup(response.text, "html.parser").find_all(class_="rankings-block__banner--rating")[0].text
  }
  batting_data.append(row)
  for batsman in soup[0].find_all(class_="table-body"):
    row = {
        'Rank': batsman.find(class_="rankings-table__pos-number").text.strip('\n').strip(' ').strip('\n'),
        'Name': batsman.find(class_="table-body__cell name").text.strip('\n').strip(' '),
        'Country': batsman.find(class_="table-body__logo-text").text,
        'Rating': batsman.find(class_="table-body__cell u-text-right rating").text
    }
    batting_data.append(row)

  df = pd.DataFrame(batting_data)
  print(df.to_string())

  bowling_data = []
  top_player_info = BeautifulSoup(response.text, "html.parser").find_all(class_= "rankings-block__banner--player-info")[1]
  row = {
      'Rank': 1,
      'Name': top_player_info.find(class_="rankings-block__banner--name").text.strip('\n').strip(' '),
      'Country': top_player_info.find(class_="rankings-block__banner--nationality").text.strip('\n').strip(' '),
      'Rating': BeautifulSoup(response.text, "html.parser").find_all(class_="rankings-block__banner--rating")[1].text
  }
  bowling_data.append(row)
  for batsman in soup[1].find_all(class_="table-body"):
    row = {
        'Rank': batsman.find(class_="rankings-table__pos-number").text.strip('\n').strip(' ').strip('\n'),
        'Name': batsman.find(class_="table-body__cell name").text.strip('\n').strip(' '),
        'Country': batsman.find(class_="table-body__logo-text").text,
        'Rating': batsman.find(class_="table-body__cell u-text-right rating").text
    }
    bowling_data.append(row)
  
  df = pd.DataFrame(bowling_data)
  print(df.to_string())

get_top_icc_team_details('https://www.icc-cricket.com/rankings/mens/player-rankings/odi')
print("=====================================================")
get_top_icc_team_details('https://www.icc-cricket.com/rankings/womens/player-rankings/odi')

           Team Matches Points Rating
0     Australia      19  2,439    128
1         India      29  3,318    114
2  South Africa      21  2,306    110
3       England      39  3,909    100
4   New Zealand      27  2,704    100
5      Pakistan      23  2,111     92
6     Sri Lanka      23  1,916     83
7   West Indies      25  1,988     80
8    Bangladesh      22  1,047     48
9      Zimbabwe       6    148     25
  Rank                   Name Country Rating
0    1             Babar Azam     PAK    892
1    2            Imam-ul-Haq     PAK    815
2    3  Rassie van der Dussen      SA    789
3    4        Quinton de Kock      SA    784
4    5            Virat Kohli     IND    767
5    6           Rohit Sharma     IND    763
6    7            Ross Taylor      NZ    744
7    8           David Warner     AUS    737
8    9         Jonny Bairstow     ENG    732
9   10            Aaron Finch     AUS    715
  Rank              Name Country Rating
0    1       Trent Boult      NZ    697
1    2 


## Q7


In [123]:
def get_news_headlines(url):

  data = []

  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser").find(id = "Home Page International-riverPlus")

  for i in range(10):
    row = {
        'News Headline': soup.find_all(class_="RiverPlusCard-container")[i].text,
        'Link': soup.find_all(class_="RiverPlusCard-container")[i].find_all('a', href=True)[-1]['href'],
        'Time': soup.find_all(class_ = "RiverByline-datePublished")[i].text
    }
    data.append(row)

  df = pd.DataFrame(data)
  print(df.to_string())

get_news_headlines('https://www.cnbc.com/world/?region=world/')

                                                                                                                            News Headline                                                                                                                                       Link         Time
0                                                The $300 billion meme stock that makes GameStop look like child's play2 hours ago•Yun Li                                  https://www.cnbc.com/2022/08/03/the-300-billion-meme-stock-that-makes-gamestop-look-like-childs-play.html  2 hours ago
1                   Pelosi's trip to Taiwan is like 'pouring salt in an open wound for China', Stephen Roach says4 hours ago•Sumathi Bala                                 https://www.cnbc.com/2022/08/03/pelosis-trip-to-taiwan-makes-matters-worse-for-us-and-china-relations.html  2 hours ago
2                                                    These stocks are poised for a comeback if inflation peaks, Jefferies saysWeiz

## Q8

In [124]:
def get_ai_articles_details(url):

  data = []
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser").find_all(class_="sc-9zxyh7-1 sc-9zxyh7-2 exAXfr jQmQZp")

  for article in soup:
    row = {
        'Name': article.find(["a"]).text,
        'Authors': article.find(["p"]).text,
        'Published Date': article.find_all("span")[-1].text,
        'Paper URL': article.find("a", href=True)['href']
    }
    data.append(row)

  df = pd.DataFrame(data)
  print(df.to_string()) 

get_ai_articles_details('https://www.journals.elsevier.com/artificial-intelligence/most-downloaded-articles')

                                                                                                                                       Name                                                                                             Authors  Published Date                                                            Paper URL
0                                                                                                                          Reward is enough           Silver, David, Singh, Satinder, Precup, Doina, Sutton, Richard S. Open AccessOctober 2021    October 2021  https://www.sciencedirect.com/science/article/pii/S0004370221000862
1                                                                                                                 Making sense of raw input                                    Evans, Richard, Bošnjak, Matko and 5 moreOpen AccessOctober 2021    October 2021  https://www.sciencedirect.com/science/article/pii/S0004370221000722
2                        

## Q9

In [125]:
def get_restaurant_details(url):

  data = []
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser").find_all(class_="restnt-card restaurant ")

  for rest in soup:
    row = {
        'Name': rest.find(class_="restnt-name ellipsis").text,
        'Cuisines': [cuisine.text for cuisine in rest.find(class_="double-line-ellipsis").find_all(["a"], href=True)],
        'Location': rest.find(class_="restnt-loc ellipsis").text,
        'Rating': rest.find(class_="restnt-rating rating-4").text,
        'Image URL': rest.find(class_="lazy-load-img no-img")['data-src']
    }
    data.append(row)

  df = pd.DataFrame(data)
  print(df.to_string())  

get_restaurant_details('https://www.dineout.co.in/delhi-restaurants/buffet-special')

                               Name                        Cuisines                                                  Location Rating                                                                                                            Image URL
0                   Castle Barbeque         [Chinese, North Indian]                            Connaught Place, Central Delhi    4.1  https://im1.dineout.co.in/images/uploads/restaurant/sharpen/8/k/b/p86792-16062953735fbe1f4d3fb7e.jpg?tr=tr:n-medium
1                   Jungle Jamboree  [North Indian, Asian, Italian]                    3CS Mall,Lajpat Nagar - 3, South Delhi    3.9   https://im1.dineout.co.in/images/uploads/restaurant/sharpen/3/h/c/p3643-144497865356209fdd65746.jpg?tr=tr:n-medium
2                   Castle Barbeque         [Chinese, North Indian]                    Pacific Mall,Tagore Garden, West Delhi    3.9  https://im1.dineout.co.in/images/uploads/restaurant/sharpen/3/j/o/p38113-15959192065f1fcb666130c.jpg?tr=tr:n-medium


## Q10

In [126]:
def get_publication_details(url):

  data = []
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser").find_all(["tr"])

  for pub in soup[1:]:
    row = {
        'Rank': pub.find_all(["td"])[0].text,
        'Publication': pub.find_all(["td"])[1].text,
        'h5-index': pub.find_all(["td"])[2].text,
        'h5-median': pub.find_all(["td"])[3].text
    }
    data.append(row)

  df = pd.DataFrame(data)
  print(df.to_string())

get_publication_details('https://scholar.google.com/citations?view_op=top_venues&hl=en')

    Rank                                                             Publication h5-index h5-median
0     1.                                                                  Nature      444       667
1     2.                                     The New England Journal of Medicine      432       780
2     3.                                                                 Science      401       614
3     4.          IEEE/CVF Conference on Computer Vision and Pattern Recognition      389       627
4     5.                                                              The Lancet      354       635
5     6.                                                      Advanced Materials      312       418
6     7.                                                   Nature Communications      307       428
7     8.                                                                    Cell      300       505
8     9.                    International Conference on Learning Representations      286       533
